In [1]:
import numpy as np
import pandas as pd

### 1. Reading raw investments data from csv file
    * Includes investment rounds in 2008-18 for companies not founded between 2008-18  as well

In [2]:
#Reading Total Investments Data for all Companies in the sector between 2008-18

df_inv = pd.read_csv('./raw_data/investmentData.csv', header=None, 
                                                    names=['Date','DateRoundCount', 'Org_Name','Funding_currency', 
                                                           'Funding_Amt', 'Funding_Round_Type', 'Investors'], 
                                                    parse_dates=['Date'])

#Dropping columns not required in analysis
df_inv.drop(axis=1, columns=['DateRoundCount','Investors'], inplace=True)

#Filling NaN values for Date column
df_inv['Date'].fillna(method='ffill',inplace=True)

In [3]:
df_inv.head(3)

,Date,Org_Name,Funding_currency,Funding_Amt,Funding_Round_Type
0,2018-12-03,Zelros,EUR,4000000,Series A
1,2018-12-03,Axyon AI,EUR,1300000,Series Unknown
2,2018-11-28,Oriente,USD,105000000,Series A


###  2. Filtering and keeping rows representing investment rounds for companies founded from 2008-18 only
      * To include investment rounds only for companies  founded between 2008-18

#### a. Fetching data on companies founding date from companyData.csv file in inputs/ folder

In [4]:
#Reading company founding details from companyData.csv file

df_founded = pd.read_csv('./raw_data/companyData.csv', usecols=['Founded Date','Organization Name'], 
                         parse_dates=['Founded Date'])

df_founded.head(3)

,Founded Date,Organization Name
0,2014-08-30,Heliocor
1,2017-04-01,Oriente
2,2015-01-01,Lemonade


#### b. Filtering entries of companies that are not founded between 2008-18 or whose founded date is not known

In [5]:
#Dropping companies whose founded date is not known

df_founded = df_founded[df_founded['Founded Date'].notna()]


# Converting 'Founded date' to 'Founded_Year' and renaming 'Organization Name' to 'Org_Name'

df_founded['Founded Date'] = pd.to_datetime(df_founded['Founded Date']).dt.year

df_founded.rename(columns={'Founded Date':'Founded_Year', 'Organization Name':'Org_Name'}, inplace=True)


#Finding companies whose founded date isn't between 2008-18

df_founded = df_founded[(df_founded.Founded_Year<2008) | (df_founded.Founded_Year>2018)]

#### c. Identifying company entries to drop from investments data ie.,  investment rounds entries of  companies founded before 2008-18

In [6]:
# Ensuring 'Org_Name' strings don't contain any spaces like 'Comp1 ' or ' Comp2 '

df_founded['Org_Name'] = df_founded['Org_Name'].str.strip()

df_inv['Org_Name'] = df_inv['Org_Name'].str.strip()

In [7]:
#Set of all companies founded before 2008

toDrop_total = set(df_founded['Org_Name'].str.strip())


# Set of all companies that have recieved investments in 2008-18

in_dfInv = set(df_inv['Org_Name'].str.strip())


# Set of all companies founded before 2008, that have not recieved any investments

toDrop_notIn_dfInv = toDrop_total - in_dfInv

#Set of all companies founded before 2008, that have recieved investments in 2008-18

toDrop_from_dfInv = toDrop_total - toDrop_notIn_dfInv

print(toDrop_from_dfInv)

{'Questel', 'Nanalyze', 'Finicity', 'Tiaxa', 'Pendo Systems', 'Dtex Systems', 'Saffron Technology', 'Crisp Thinking', 'China Rapid Finance', 'IDV Solutions', 'THRON - The Intelligent DAM for Marketers.', 'Aggregate Knowledge'}


#### d. Dropping company entries outside analysis period from dataframe

In [8]:
# Dropping all companies founded before 2008 and after 2018, that have recieved investments between 2008-18


for company in toDrop_from_dfInv:
    
    df_inv.drop(axis=0, index=df_inv.index[df_inv['Org_Name']==company],inplace=True)


### 3. Converting Funding Amounts in diff currencies to USD  allow  for generalized summation and comparision
        *Adding  ( x + y + z + ... ) USD  as adding ( x EUR + y USD + z INR ) based only on x,y,z values isn't possible
    

In [9]:
# Replacing undisclosed amounts by zero and currency of undisclosed amounts by USD

df_inv['Funding_Amt'].replace('undisclosed',0,inplace=True)

df_inv['Funding_currency'].replace({'NA':'USD', None:'USD'},inplace=True)

#### i Currency Conversion to USD

    Using package CurrencyConverter (ref https://pypi.org/project/CurrencyConverter/)
    
    The data source is the European Central Bank. This is the ECB historical rates for 42 currencies against the Euro since 1999. 

In [10]:
#Importing currency converter and instantiating CurrencyConverter object with input as exchange rates csv file

from currency_converter import CurrencyConverter

c = CurrencyConverter('./input_data/currency_exchange_rates/exRateHist.csv')


#Creating 'Funding_Amt_USD' column in df_inv dataframe

df_inv['Funding_Amt_USD']=0

In [11]:
# Ensuring 'Funding_currency' strings don't contain any spaces like 'EUR ' or ' INR '

df_inv['Funding_currency'] = df_inv['Funding_currency'].str.strip()

In [12]:
#Converting all Funding Amounts to USD

df_inv['Funding_Amt_USD'] = df_inv.apply( lambda x : int( c.convert( x['Funding_Amt'], 
                                                                     x['Funding_currency'],
                                                                     'USD', x['Date'] 
                                                                   )
                                                        ),             
                                          axis=1
                                        )            

In [13]:
df_inv.head(3)

,Date,Org_Name,Funding_currency,Funding_Amt,Funding_Round_Type,Funding_Amt_USD
0,2018-12-03,Zelros,EUR,4000000,Series A,4532800
1,2018-12-03,Axyon AI,EUR,1300000,Series Unknown,1473160
2,2018-11-28,Oriente,USD,105000000,Series A,105000000


#### ii Dropping Funding Amounts in different currencies

In [14]:
#Dropping columns 'Funding_currency' and 'Funding_Amt'

df_inv.drop(columns=['Funding_currency', 'Funding_Amt'], inplace=True)

In [15]:
df_inv.head(3)

,Date,Org_Name,Funding_Round_Type,Funding_Amt_USD
0,2018-12-03,Zelros,Series A,4532800
1,2018-12-03,Axyon AI,Series Unknown,1473160
2,2018-11-28,Oriente,Series A,105000000


### 4. Converting Funding Date to Funding Year

In [16]:
#Converting 'Date' column to 'Year' column

df_inv['Date'] = pd.to_datetime(df_inv['Date']).dt.year

df_inv.rename(columns={'Date':'Year'}, inplace=True)

# Resetting index

df_inv.reset_index(drop=True,inplace=True)

In [17]:
df_inv.head(3)

,Year,Org_Name,Funding_Round_Type,Funding_Amt_USD
0,2018,Zelros,Series A,4532800
1,2018,Axyon AI,Series Unknown,1473160
2,2018,Oriente,Series A,105000000


 ### 5. Adding Funding Stage Column

#### Funding Stage column categorizes Funding Round Types under different stages as follows

<p><b>Stage</b> : Round Types </p>
<p><b>Seed</b> : Angel, Pre-seed, Seed, Grant, Non-Equity Assistance, Convertible Note</p>
<p><b>Early Stage Venture</b>: 	Series A, Series B</p>
<p><b>Late stage venture</b>: 	Series C, Seires D, Series E, Series F, Corporate Round, Secondary Market</p>
<p><b>Private Equity</b>:	Private Equity</p>
<p><b>Uncategorized (NA)</b> :Venture-Series Unknown, Undisclosed, Funding Round, Initial Coin Offering, Equity Crowdfunding, Product Crowdfunding, Debt Financing</p>

### a. Set of All Funding Round Types present

In [18]:
print(df_inv['Funding_Round_Type'].unique())

['Series A' 'Series Unknown' 'Seed' 'Series B' 'Grant' 'Pre Seed'
 'Initial Coin Offering' 'Funding Round' 'Convertible Note' 'Series C'
 'Angel' 'Secondary Market' 'Equity Crowdfunding' 'Debt Financing'
 'Series D' 'Non Equity Assistance' 'Product Crowdfunding'
 'Private Equity' 'Corporate Round']


### b. Creating Funding Stage Column and Dictionary of Funding Stage &  Rounds under each each Stage

In [19]:
# Creating Funding_Stage Column
df_inv['Funding_Stage'] = 0

# Creating stageLst with  key as Funding Stage and value as List of Funding Rounds categorized under the Funding Stage
stageLst = {
    
             'Seed' : ['Angel', 'Pre Seed', 'Seed', 'Grant', 'Non Equity Assistance', 'Convertible Note'],

             'Early Stage': ['Series A', 'Series B'],

             'Late Stage': ['Series C', 'Series D','Series E', 'Series F','Corporate Round', 'Secondary Market'],

            'Private Equity': ['Private Equity'],

            'Uncategorized': ['Series Unknown', 'Initial Coin Offering', 'Funding Round',
                         
                              'Equity Crowdfunding','Product Crowdfunding','Debt Financing']
            }


### Assigning values to Funding Stage Column based on dictionary { Stage : [Funding Rounds] }

In [20]:

for key, valueLst in stageLst.items():
    
    df_inv.loc[df_inv['Funding_Round_Type'].isin(valueLst),'Funding_Stage']=key
    

In [21]:
df_inv.head(3)

,Year,Org_Name,Funding_Round_Type,Funding_Amt_USD,Funding_Stage
0,2018,Zelros,Series A,4532800,Early Stage
1,2018,Axyon AI,Series Unknown,1473160,Uncategorized
2,2018,Oriente,Series A,105000000,Early Stage


### 6. Saving to investments input csv file

In [22]:
#Saving to file

df_inv.to_csv('./input_data/investmentInput.csv', index=False)

### 7. Deleting dataframes to deallocate memory

In [23]:
del df_inv
del df_founded